<a href="https://colab.research.google.com/github/MinnDdu/deep_learning_practice/blob/main/cat_dog_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os

os.environ['KAGGLE_CONFIG_DIR'] = '/content/'

!kaggle competitions download -c dogs-vs-cats-redux-kernels-edition

 98% 796M/814M [00:07<00:00, 124MB/s]
100% 814M/814M [00:07<00:00, 114MB/s]


In [3]:
!unzip -q dogs-vs-cats-redux-kernels-edition.zip

In [4]:
!unzip -q train.zip -d .

In [5]:
import os
import tensorflow as tf
import shutil

print(len(os.listdir('/content/train/')))
# os.listdir() -> list안에 파일들을 담아줌 [파일명1, 파일명2,...]

# bug fix - colab에선 파이썬 명령어로 경로 만들어야 클래스가 하나 더 생기는 일 방지
os.mkdir('/content/dataset')
os.mkdir('/content/dataset/cat')
os.mkdir('/content/dataset/dog')

# 파일을 숫자로 변환
# 1. opencv library로 반복문으로 이미지 숫자화
# 2. tf.keras 이용해서 한번에 처리
for i in os.listdir('/content/train/'):
    # i는 'cat01.jpg' ...
    # shutil.copyfile('어떤경로의 파일을', '어떤 경로에다가 복사')
    if 'cat' in i:
        shutil.copyfile('/content/train/' + i, '/content/dataset/cat/' + i)
    if 'dog' in i:
        shutil.copyfile('/content/train/' + i, '/content/dataset/dog/' + i)





25000


In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/dataset/', 
    image_size=(64,64), 
    batch_size=32, # 이미지 전부 한번에 epoch에 넣지 않고 batch 숫자만큼 넣고 w계산, 갱신
    subset='training',
    validation_split=0.2, # 데이터의 80%를 training dataset으로 감
    seed=1234
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/dataset/', 
    image_size=(64,64), 
    batch_size=32, # 이미지 전부 한번에 epoch에 넣지 않고 batch 숫자만큼 넣고 w계산, 갱신
    subset='validation',
    validation_split=0.2, # 데이터의 20%를 validation dataset으로 감
    seed=1234
)
# train_ds의 결과 -> ((xxxxxx... - 이미지 숫자화됨), (yyyyyy... - 0또는1))

# image classification 에선 그 전에 파일들을 나누어 눠야함 
# Ex) cat file, dog file 만들고 그 사진들 넣기 - 이미지 분류 딥러닝 국룰

print(train_ds)

# tuning - 레이어 더 늘리기, 
# 이미지(데이터) 전처리가 중요! -> 데이터 양 올리기 or 데이터 질 올리기
def preprocessing(i, answer):
    i = tf.cast(i/255.0, tf.float32)
    return i, answer

train_ds = train_ds.map(preprocessing)
val_ds = train_ds.map(preprocessing)


import matplotlib.pyplot as plt

for i, answer in train_ds.take(1): 
    # train_ds는 batch dataset이라는 일종의 자료형 take(1) -> 하나의 batch 가져옴
    print(i)
    print(answer)
    # plt.imshow(i[0].numpy().astype('uint8')) # i[0]은 tensor -> numpy()로 넘파이형으로 캐스팅
    # plt.show()





In [26]:
# 딥러닝 모델 만들기

# Image Augmentation - 이미지 증강 -> 기존 이미지 데이터를 약간 확대/이동/뒤집기 해서 새로운 데이터같이 이용
# 방법1 -> 증강된 데이터 사본 생성
# 방법2 -> 모델에 넣기 전에 이미지 증강

model = tf.keras.Sequential([
    # 모델에 넣기 전에 이미지 증강
    tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal', input_shape=(64, 64, 3)), # 확률로 데이터 가로로 뒤집기
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.1), # 데이터 회전
    tf.keras.layers.experimental.preprocessing.RandomZoom(0.1), # 데이터 줌
    # Ex) epoch이 10 -> 기존) 같은데이터 10번 / 증강) 약간다른 데이터 10번

    tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),

    tf.keras.layers.Dropout(0.2), # overfitting 완화 -> dropout 레이어 - 윗레이어 노드를 일부 제거
    
    tf.keras.layers.Conv2D(128, (3, 3), padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),

    tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2), # overfitting 완화 -> dropout 레이어 - 윗레이어 노드를 일부 제거
    tf.keras.layers.Dense(1, activation='sigmoid') # binary crossentropy는 마지막 sigmoid 필요
])

model.summary()

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_ds, validation_data=val_ds, epochs=5)
# train_ds ((이미지들), (정답들)) 형태
    





Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 random_flip_1 (RandomFlip)  (None, 64, 64, 3)         0         
                                                                 
 random_rotation_1 (RandomRo  (None, 64, 64, 3)        0         
 tation)                                                         
                                                                 
 random_zoom_1 (RandomZoom)  (None, 64, 64, 3)         0         
                                                                 
 conv2d_8 (Conv2D)           (None, 64, 64, 32)        896       
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 32, 32, 32)       0         
 2D)                                                             
                                                                 
 dropout_8 (Dropout)         (None, 32, 32, 32)       